In [1]:
from sympy import *
import numpy as np
from itertools import combinations
init_printing(use_unicode=True)

In [2]:
# Define functions for my non-commutative Kirillov formula

# Coefficient relying on n, j, m
def coefficient_factor(n: int, j: int, m: int):
    return factorial(n-1) / factorial(n-j+m-1)

# Binomial Factor
def binomial_factor(j: int, m: int):
    return binomial(j,m)

# Recursive function of rdr
def dr_recursion(p: int, dr = symbols('dr') ):
    summation = 0
    recur = 0
    for q in range(p+1):
        if q == 0:
            recur = 1
        else:
            recur = dr_recursion(q-1, dr)
        summation += binomial_factor(p,q) * (-1)**(p-q) * factorial(p-q) * dr * recur
    return summation

# Combination function
def all_draws(input_list, k):
    return list(combinations(input_list, k))

# Function to compute principal minors of degree n
def principal_minors_k(matrix, k):
    
    size = matrix.shape[0]  # Get the size of the square matrix
    minors = []

    if k == 0:
        return [1]
    elif k == size:
        return [det(matrix)]
    elif k > size:
        return "k is larger than size of the matrix"

    # define the combinations of row/col to delete
    input_list = [i for i in range(size)]
    combos = all_draws(input_list, size-k)

    for comb in combos:
        #print(comb)
        submatrix = matrix.copy()   # Make temp_matrix that is mutable
        temp = np.delete(np.delete(submatrix, comb, axis=0), comb, axis=1)
        #display(Matrix(temp))
        minors.append(det(Matrix(temp))) 
    return minors
    
# Main function
def non_comm_kirillov(dpi: list, n: int):
    dr = symbols('dr')
    formula = np.zeros((n, n))
    matrix = np.zeros((n, n))
    recur = 0
    for k in range((n-1)+1):
        for j in range((n-k-1)+1):
            for m in range((j)+1):
                if k == 0:
                    matrix = Matrix(np.eye(n))
                else:
                    matrix = Matrix(dpi)**k
                if m == 0:
                    recur = 1
                else:
                    recur = dr_recursion(m-1, dr)
                
                formula += (-1)**(n-k-j-1) * binomial_factor(j, m) * coefficient_factor(n, j, m) * \
                matrix * sum(principal_minors_k(Matrix(dpi), n-j-k-1)) * recur
    return 1/factorial(n-1) * formula

In [23]:
# Define differential operation functions

# SU(2)case
def dY_alpha (f, h, x, y):
    h, x, y = symbols('h, x, y')
    return (I*x - y)*diff(f,h) - h * (I * diff(f,x) - diff(f,y))

In [12]:
# define the symbols
h, x, y = symbols('h, x, y')
matrix = [[I*h, x + I*y], [-x + I*y, -I*h]]
matrix2 = [[2*I * h, sqrt(2)*(x + I*y), 0],[-sqrt(2)*(x - I*y), 0, sqrt(2)*(x + I*y) ],[0, -sqrt(2)*(x - I*y), -2* I * h]]
#matrix = ImmutableMatrix(matrix)
#matrix2 = ImmutableMatrix(matrix2)
matrix = Matrix(matrix)
matrix2 = Matrix(matrix2)
display(matrix2)

⎡    2⋅ⅈ⋅h      √2⋅(x + ⅈ⋅y)        0      ⎤
⎢                                          ⎥
⎢-√2⋅(x - ⅈ⋅y)        0        √2⋅(x + ⅈ⋅y)⎥
⎢                                          ⎥
⎣      0        -√2⋅(x - ⅈ⋅y)     -2⋅ⅈ⋅h   ⎦

In [24]:
#non_comm_kirillov(matrix, 2) SU(2)
display(matrix)
result_matrix = non_comm_kirillov(matrix, 2)
expand(result_matrix)
pi11 = expand(result_matrix)[0,0]
pi11 = nsimplify(pi11)
display(result_matrix)
expand(dY_alpha(-pi11, h, x, y))

⎡  ⅈ⋅h     x + ⅈ⋅y⎤
⎢                 ⎥
⎣-x + ⅈ⋅y   -ⅈ⋅h  ⎦

⎡1.0⋅dr + ⅈ⋅h + 1.0       x + ⅈ⋅y      ⎤
⎢                                      ⎥
⎣     -x + ⅈ⋅y       1.0⋅dr - ⅈ⋅h + 1.0⎦

In [25]:
#non_comm_kirillov(matrix2, 3) SU(2)
display(matrix2)
result_matrix = non_comm_kirillov(matrix2, 3)
pi11 = expand(result_matrix)[0,0]
pi12 = expand(result_matrix)[0,1]
display(pi12)
pi11 = nsimplify(pi11)
display(result_matrix)
expand(dY_alpha(-pi12, h, x, y))

⎡    2⋅ⅈ⋅h      √2⋅(x + ⅈ⋅y)        0      ⎤
⎢                                          ⎥
⎢-√2⋅(x - ⅈ⋅y)        0        √2⋅(x + ⅈ⋅y)⎥
⎢                                          ⎥
⎣      0        -√2⋅(x - ⅈ⋅y)     -2⋅ⅈ⋅h   ⎦

⎡       2                              2    2           √2⋅dr⋅(x + ⅈ⋅y)                                   
⎢ 0.5⋅dr  + ⅈ⋅dr⋅h + 1.5⋅dr + 2⋅ⅈ⋅h + x  + y  + 1.0     ─────────────── + √2⋅ⅈ⋅h⋅x - √2⋅h⋅y + √2⋅(x + ⅈ⋅y)
⎢                                                              2                                          
⎢                                                                                                         
⎢  √2⋅dr⋅(x - ⅈ⋅y)                                                     2                 2                
⎢- ─────────────── - √2⋅ⅈ⋅h⋅x - √2⋅h⋅y - √2⋅(x - ⅈ⋅y)            0.5⋅dr  + 1.5⋅dr + 2.0⋅h  + 1.0          
⎢         2                                                                                               
⎢                                                                                                         
⎢                  2              2                      √2⋅dr⋅(x - ⅈ⋅y)                                  
⎢                 x  - 2⋅ⅈ⋅x⋅y - y   

In [36]:
#non_comm_kirillov(matrix2, 3) B2
h1, h2, x1, y1, x2, y2, x3, y3, x4, y4 = symbols('h1, h2, x1, y1, x2, y2, x3, y3, x4, y4')
matrix_b2 = [[I*h1, x1 + I*y1, sqrt(2)*(x3 +I*y3), -sqrt(2)*(x4 +I*y4), 0],  \
            [-x1 + I*y1, -I*h1 + 2*I*h2, sqrt(2)*(x2 +I*y2), 0, -sqrt(2)*(x4 +I*y4)], \
            [-sqrt(2)*(x3 - I*y3), -sqrt(2)*(x2 - I*y2), 0, sqrt(2)*(x2 +I*y2), -sqrt(2)*(x3 +I*y3)], \
            [sqrt(2)*(x4 - I*y4), 0, -sqrt(2)*(x2 - I*y2), I*h1 - 2*I*h2, x1 + I*y1], \
            [0, sqrt(2)*(x4 - I*y4), sqrt(2)*(x3  - I*y3), -x1 + I*y1, -I*h1]  \
           ]
matrix_b2 = Matrix(matrix_b2)
display(matrix_b2)

⎡     ⅈ⋅h₁           x₁ + ⅈ⋅y₁     √2⋅(x₃ + ⅈ⋅y₃)   -√2⋅(x₄ + ⅈ⋅y₄)         0       ⎤
⎢                                                                                   ⎥
⎢  -x₁ + ⅈ⋅y₁     -ⅈ⋅h₁ + 2⋅ⅈ⋅h₂   √2⋅(x₂ + ⅈ⋅y₂)          0         -√2⋅(x₄ + ⅈ⋅y₄)⎥
⎢                                                                                   ⎥
⎢-√2⋅(x₃ - ⅈ⋅y₃)  -√2⋅(x₂ - ⅈ⋅y₂)         0         √2⋅(x₂ + ⅈ⋅y₂)   -√2⋅(x₃ + ⅈ⋅y₃)⎥
⎢                                                                                   ⎥
⎢√2⋅(x₄ - ⅈ⋅y₄)          0         -√2⋅(x₂ - ⅈ⋅y₂)   ⅈ⋅h₁ - 2⋅ⅈ⋅h₂      x₁ + ⅈ⋅y₁   ⎥
⎢                                                                                   ⎥
⎣       0         √2⋅(x₄ - ⅈ⋅y₄)   √2⋅(x₃ - ⅈ⋅y₃)     -x₁ + ⅈ⋅y₁          -ⅈ⋅h₁     ⎦

In [52]:
#Non_comm_kirillov(matrix2, 3) B2 , 5-dimensional
result_matrix = non_comm_kirillov(matrix_b2, 5)

In [53]:
# Display the first row of the highest weight representation
pi11 = expand(result_matrix)[0,0]
pi12 = expand(result_matrix)[0,1]
pi13 = expand(result_matrix)[0,2]
pi14 = expand(result_matrix)[0,3]
pi15 = expand(result_matrix)[0,4]

pi11 = nsimplify(pi11)
pi12 = nsimplify(pi12)
pi13 = nsimplify(pi13)
pi14 = nsimplify(pi14)
pi15 = nsimplify(pi15)

display(pi11)
display(pi12)
display(pi13)
display(pi14)
display(pi15)

#display(result_matrix)
#expand(dY_alpha(-pi12, h, x, y))

In [80]:
# Define the differential operator dY_alpha1
# Spin(5) case
def dY_alpha1 (f, h1, h2, x1, y1, x2, y2, x3, y3, x4, y4):
    h1, h2, x1, y1, x2, y2, x3, y3, x4, y4 = symbols('h1, h2, x1, y1, x2, y2, x3, y3, x4, y4')
    return (y1 - I*x1)*diff(f,h1) + (h2 - h1)*diff(f,y1) - I*(h2 - h1)*diff(f,x1) + Rational(1/2) * (x2 - I*y2)*diff(f,x3) \
            + Rational(1/2) * (y2 + I*x2)*diff(f,y3) - Rational(1/2) * (x3 + I*y3)*diff(f,x2) - Rational(1/2) * (y3 - I*x3)*diff(f,y2)
    
def dY_alpha2 (f, h1, h2, x1, y1, x2, y2, x3, y3, x4, y4):
    h1, h2, x1, y1, x2, y2, x3, y3, x4, y4 = symbols('h1, h2, x1, y1, x2, y2, x3, y3, x4, y4')
    return Rational(1/2) * ( (2* y2 - 2*I*x2)*diff(f,h2) + (h1 - 2*h2)*diff(f,y2) + I*(-h1 + 2* h2)*diff(f,x2) + (2*x3 + 2*I*y3)*diff(f,x1) \
            + (2*y3 - 2*I*x3)*diff(f,y1) + ( (-x1 - sqrt(2)*x4) + I*(y1 - sqrt(2)*y4) )*diff(f,x3) \
                  + ( (-y1 - sqrt(2)*y4) + I*(-x1 + sqrt(2)*x4) )*diff(f,y3) + (sqrt(2)*x3 - sqrt(2)*I*y3) * diff(f,x4) \
                  + (sqrt(2)*y3 + sqrt(2)*I*x3) * diff(f,y4) )

In [84]:
expr = nsimplify( expand( dY_alpha1(pi14, h1, h2, x1, y1, x2, y2, x3, y3, x4, y4) ) ) 
display(expr)
display(pi15)

In [83]:
expand(expr - nsimplify(pi15))

In [3]:
#Non_comm_kirillov(matrix2, 3) A2 Adjoint representation
h1, h2, x1, y1, x2, y2, x3, y3 = symbols('h1, h2, x1, y1, x2, y2, x3, y3')
matrix_a2_aj = [[I*(h1+h2), x1+I*y1, x2+I*y2, x3+I*y3, -(x3+I*y3), 0, 0, 0],  \
               [-(x1-I*y1), I*(2*h2-h1), 0, 2*(x2+I*y2), x2+I*y2, 0, -(x3+I*y3), 0], \
               [-(x2-I*y2), 0, I*(2*h1-h2), x1+I*y1, 2*(x1+I*y1), x3+I*y3, 0, 0], \
               [-(x3-I*y3), -(x2-I*y2), 0, 0, 0, x2+I*y2, 0, -(x3+I*y3)], \
               [x3-I*y3, 0, -(x1-I*y1), 0, 0, 0, x1+I*y1, x3+I*y3], \
               [0, 0, -(x3-I*y3), -2*(x2-I*y2), -(x2-I*y2), I*(h1-2*h2), 0, x1+I*y1], \
               [0, x3-I*y3, 0, -(x1-I*y1), -2*(x1-I*y1), 0, I*(h2-2*h1), x2+I*y2], \
               [0, 0, 0, x3-I*y3, -(x3-I*y3), -(x1 - I*y1), -(x2 - I*y2), -I*(h1+h2)] \
               ]
matrix_a2_aj = Matrix(matrix_a2_aj)
display(matrix_a2_aj)

⎡ⅈ⋅(h₁ + h₂)    x₁ + ⅈ⋅y₁       x₂ + ⅈ⋅y₂      x₃ + ⅈ⋅y₃       -x₃ - ⅈ⋅y₃     
⎢                                                                             
⎢-x₁ + ⅈ⋅y₁   ⅈ⋅(-h₁ + 2⋅h₂)        0        2⋅x₂ + 2⋅ⅈ⋅y₂     x₂ + ⅈ⋅y₂      
⎢                                                                             
⎢-x₂ + ⅈ⋅y₂         0         ⅈ⋅(2⋅h₁ - h₂)    x₁ + ⅈ⋅y₁     2⋅x₁ + 2⋅ⅈ⋅y₁    
⎢                                                                             
⎢-x₃ + ⅈ⋅y₃     -x₂ + ⅈ⋅y₂          0              0               0          
⎢                                                                             
⎢ x₃ - ⅈ⋅y₃         0          -x₁ + ⅈ⋅y₁          0               0          
⎢                                                                             
⎢     0             0          -x₃ + ⅈ⋅y₃    -2⋅x₂ + 2⋅ⅈ⋅y₂    -x₂ + ⅈ⋅y₂    ⅈ
⎢                                                                             
⎢     0         x₃ - ⅈ⋅y₃           0          -x₁ +

In [5]:
# Define the differential operator dY_alpha1, dY_alpha2 
# SU(3) case
def dY_alpha1 (f, h1, h2, x1, y1, x2, y2, x3, y3):
    h1, h2, x1, y1, x2, y2, x3, y3 = symbols('h1, h2, x1, y1, x2, y2, x3, y3')
    return Rational(1/2)* ( (2*y1 - I*2*x1)*diff(f, h1) - (2*h1 - h2)*diff(f, y1) + I*(2*h1-h2)*diff(f, x1) + (x2-I*y2)*diff(f, x3) \
            + (y2 + I*x2)*diff(f, y3) + (-x3 - I*y3)*diff(f, x2) + (-y3 + I*x3)*diff(f, y2) )
    
def dY_alpha2 (f, h1, h2, x1, y1, x2, y2, x3, y3):
    h1, h2, x1, y1, x2, y2, x3, y3 = symbols('h1, h2, x1, y1, x2, y2, x3, y3')
    return Rational(1/2)* ( (2*y2 - I*2*x2)*diff(f, h2) + (h1 - 2*h2)*diff(f, y2) - I*(h1-2*h2)*diff(f, x2) + (x3+I*y3)*diff(f, x1) \
            + (y3 - I*x3)*diff(f, y1) + (-x1 + I*y1)*diff(f, x3) + (-y1 - I*x1)*diff(f, y3) )

In [ ]:
result_matrix = non_comm_kirillov(matrix_a2_aj, 8)

In [49]:
dr = symbols('dr')

for i in range(5):
    expr = expand(dr_recursion(i,dr))
    display(expr)